In [6]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\piggy\Desktop\work\AI574-project\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\piggy\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. F

In [24]:
sentences = ['Python, Java, SQL', 'Java, Python, SQL', 'Haskell, C++, Rust', 'Nursing, Medicine, Health', 'Python, Java, SQL, C++']

In [25]:
embeddings = model.encode(sentences, convert_to_tensor=True)

In [26]:
embeddings

tensor([[ 0.0082, -0.0190, -0.0605,  ...,  0.0702,  0.1088,  0.0033],
        [ 0.0109, -0.0202, -0.0627,  ...,  0.0659,  0.1015,  0.0052],
        [-0.1255,  0.0266,  0.0022,  ..., -0.0444,  0.0634,  0.0326],
        [ 0.0250, -0.0553, -0.0197,  ..., -0.0385,  0.1462,  0.0934],
        [-0.0507, -0.0218, -0.0417,  ..., -0.0199,  0.1283,  0.0363]])

In [27]:
cosine_similarity([embeddings[0]], [embeddings[1]])

array([[0.9963145]], dtype=float32)

In [30]:
cosine_similarity([embeddings[0]], [embeddings[4]])

array([[0.8436446]], dtype=float32)

In [28]:
cosine_similarity([embeddings[0]], [embeddings[2]])

array([[0.17740303]], dtype=float32)

In [29]:
cosine_similarity([embeddings[0]], [embeddings[3]])

array([[0.20144998]], dtype=float32)

In [23]:
cosine_similarity([embeddings[3]], [embeddings[2]])

array([[0.13691956]], dtype=float32)